In [1]:
!rm -rf torch_compile_debug
!rm *.svg

In [ ]:
import torch
import math
import os
import matplotlib.pyplot as plt
import torch._dynamo
from torchvision import models
from torch.fx.passes.graph_drawer import FxGraphDrawer
from IPython.display import Markdown as md

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [ ]:
def f(x):
    return torch.sin(x)**2 + torch.cos(x)**2

md('''
# $ y = f(x) = sin^2(x) + cos^2(x)$
''')

In [ ]:
md('''
## Optimization problem:
### find $w^*$ that $\displaystyle \min_{w} f(w)$

## Gradient update
### $ w_{i+1} = w_{i} - g(\\nabla f(w))$
## For SGD
### $  g(\\nabla f(w)) = \\alpha*\\nabla f(w)$
## Which makes the update for SGD:
### $ w_{i+1} = w_{i} - \\alpha*\\nabla f(w)$

## Loss function
### $loss(w): loss(model(w,batch_{inputs}), batch_{outputs})$ 
''')

In [ ]:
md('''
# **Forward graph:** $f(x) = sin^2(x)+cos^2(x)$ \n
# **Backward graph:** $\\frac {df(x)}{d\\vec{w}} = f\'(x) = 2sin(x)cos(x) + 2cos(x)(-sin(x))$
''')

In [ ]:
torch.manual_seed(0)
x = torch.rand(1000, requires_grad=True).to(device)
torch.nn.functional.mse_loss(f(x),torch.ones_like(x)) < 1e-10

In [ ]:
torch.manual_seed(0)
x = torch.rand(1000, requires_grad=True).to(device)

compiled_f = torch.compile(f)
torch.nn.functional.mse_loss(compiled_f(x),torch.ones_like(x)) < 1e-10

In [ ]:
def inspect_backend(gm, sample_inputs):
    code = gm.print_readable()
    with open("forward.svg", "wb") as file:
        file.write(FxGraphDrawer(gm,'f').get_dot_graph().create_svg())
    return gm.forward

torch._dynamo.reset()
compiled_f = torch.compile(f, backend=inspect_backend)

x = torch.rand(1000, requires_grad=True).to(device)
out = compiled_f(x)

md(f'''
### Graph
![]({'forward.svg'})
''')

# AOTAutograd and Aten IR

In [ ]:
import torch._dynamo
from torch.fx.passes.graph_drawer import FxGraphDrawer
from functorch.compile import make_boxed_func
from torch._functorch.aot_autograd import aot_module_simplified

def f(x):
    return torch.sin(x)**2 + torch.cos(x)**2

def inspect_backend(gm, sample_inputs): 
    # Forward compiler capture
    def fw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("forward_aot.svg", "wb") as file:
            file.write(g.get_dot_graph().create_svg())
        return make_boxed_func(gm.forward)
    
    # Backward compiler capture
    def bw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("backward_aot.svg", "wb") as file:
            file.write(g.get_dot_graph().create_svg())
        return make_boxed_func(gm.forward)
    
    # Call AOTAutograd
    gm_forward = aot_module_simplified(gm,sample_inputs,
                                       fw_compiler=fw,
                                       bw_compiler=bw)

    return gm_forward

torch.manual_seed(0)
x = torch.rand(1000, requires_grad=True).to(device)
y = torch.ones_like(x)

torch._dynamo.reset()
compiled_f = torch.compile(f, backend=inspect_backend)
out = torch.nn.functional.mse_loss(compiled_f(x), y).backward()

In [ ]:
md(f'''
|![]({'forward_aot.svg'}) | < Forward graph <br><br><br> Backward graph >|![]({'backward_aot.svg'})|
|---|---|---|
''')

# Decomposition to Core Aten IR

In [ ]:
import torch._dynamo
from torch.fx.passes.graph_drawer import FxGraphDrawer
from functorch.compile import make_boxed_func
from torch._functorch.aot_autograd import aot_module_simplified
from torch._decomp import core_aten_decompositions

def f_loss(x, y):
    f_x = torch.sin(x)**2 + torch.cos(x)**2
    return torch.nn.functional.mse_loss(f_x, y)

# decompositions = core_aten_decompositions() # Use decomposition to Core Aten IR
decompositions = {} # Don't use decomposition to Core Aten IR

def inspect_backend(gm, sample_inputs): 
    def fw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("forward_decomp.svg", "wb") as file:
            file.write(g.get_dot_graph().create_svg())
        return make_boxed_func(gm.forward)
    
    def bw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("backward_decomp.svg", "wb") as file:
            file.write(g.get_dot_graph().create_svg())
        return make_boxed_func(gm.forward)

    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_inputs,
        fw_compiler=fw,
        bw_compiler=bw,
        decompositions=decompositions
    )

torch.manual_seed(0)
x = torch.rand(1000, requires_grad=True).to(device)
y = torch.ones_like(x)

torch._dynamo.reset()
compiled_f = torch.compile(f_loss, backend=inspect_backend)
out = compiled_f(x,y).backward()


md('''
# $MSE = (\\frac{1}{n})(\\vec{y}-\\vec{x})^2$

''')

In [ ]:
md(f'''
|![]({'forward_decomp.svg'}) | < Forward graph <br><br><br> Backward graph >|![]({'backward_decomp.svg'})|
|---|---|---|
''')

# Decomposition to prim IR

In [ ]:
import torch._dynamo
from torch.fx.passes.graph_drawer import FxGraphDrawer
from functorch.compile import make_boxed_func
from torch._functorch.aot_autograd import aot_module_simplified
from torch._decomp import core_aten_decompositions

def f_loss(x, y):
    f_x = torch.sin(x)**2 + torch.cos(x)**2
    return torch.nn.functional.mse_loss(f_x, y)

decompositions = core_aten_decompositions()
decompositions.update(
    torch._decomp.get_decompositions([
        torch.ops.aten.sin,
        torch.ops.aten.cos,
        torch.ops.aten.add,
        torch.ops.aten.sub,
        torch.ops.aten.mul,
        torch.ops.aten.sum,
        torch.ops.aten.mean,
        torch.ops.aten.pow.Tensor_Scalar,
    ])
)

def inspect_backend(gm, sample_inputs): 
    def fw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("forward_decomp_prims.svg", "wb") as f:
            f.write(g.get_dot_graph().create_svg())
        return make_boxed_func(gm.forward)
    
    def bw(gm, sample_inputs):
        gm.print_readable()
        g = FxGraphDrawer(gm, 'fn')
        with open("backward_decomp_prims.svg", "wb") as f:
            f.write(g.get_dot_graph().create_svg())
        return make_boxed_func(gm.forward)

    # Invoke AOTAutograd
    return aot_module_simplified(
        gm,
        sample_inputs,
        fw_compiler=fw,
        bw_compiler=bw,
        decompositions=decompositions
    )

torch.manual_seed(0)
x = torch.rand(1000, requires_grad=True).to(device)
y = torch.ones_like(x)

torch._dynamo.reset()
compiled_f = torch.compile(f_loss, backend=inspect_backend)
out = compiled_f(x,y).backward()


In [ ]:
md(f'''
|![]({'forward_decomp_prims.svg'}) | < Forward graph <br><br><br> Backward graph >|![]({'backward_decomp_prims.svg'})|
|---|---|---|
''')

In [ ]:
def f(x):
    return torch.sin(x)**2 + torch.cos(x)**2 

torch._dynamo.reset()
compiled_f = torch.compile(f, backend='inductor',
                              options={'trace.enabled':True,
                                       'trace.graph_diagram':True})


# device = 'cpu'
device = 'cuda'

torch.manual_seed(0)
x = torch.rand(1000, requires_grad=True).to(device)
y = torch.ones_like(x)

out = torch.nn.functional.mse_loss(compiled_f(x),y).backward()

In [ ]:
import glob
fwd = glob.glob('torch_compile_debug/run_*/aot_torchinductor/*forward*/graph_diagram.svg')[-1]
bwd = glob.glob('torch_compile_debug/run_*/aot_torchinductor/*backward*/graph_diagram.svg')[-1]

md(f'''
|![]({fwd}) | < Forward graph <br><br><br> Backward graph >|![]({bwd})|
|---|---|---|
''')